# ```stockPrice.ipynb```
- KRX KOSPI200의 구성 종목들에 대한 파일
- 아래 파일의 데이터를 수집, 전처리한 파일
  - ```components_list.csv```
  - ```stockPrice.csv```

# import

In [1]:
import os
import sys
import time
import pickle
import warnings
import urllib.request
from glob import glob

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import koreanize_matplotlib
import requests
from bs4 import BeautifulSoup as bs

import FinanceDataReader as fdr
from tqdm import tqdm


warnings.filterwarnings('ignore')
pd.options.display.max_columns = None
# pd.options.display.float_format = '{:.4f}'.format
plt.style.use("ggplot")
%config InlineBackend.figure_format = 'retina'


sys.path.append("../import")
import module as m

data_path = m.data_path
fp_stock = f"""{m.fp["stockPrice"]}"""
fp_components = f"{data_path}components_list.csv"

data_path : ../data/
fp
{'esgRating': '../data/esgRating.parquet',
 'finaStat': '../data/finaStat.parquet',
 'stockPrice': '../data/stockPrice.parquet',
 'stockPrice_year': '../data/stockPrice_year.parquet'}


# ```components_list.csv```
  - 개요
    - KRX의 KOSPI200지수 구성 종목 리스트
      - components : 구성 요소를 의미
    - 분석의 대상을 추려 내기 위함.
  - 설명
    - 2010-06-31~2022-06-31 기간
    - 매년 06월 말일과 12월 말일에 대한 데이터
  - Data Collection Method
    - 직접 다운로드
    - Data Source
      - [KRX (만료된 URL)](http://index.krx.co.kr/contents/MKD/03/0304/03040101/MKD03040101.jsp?idxCd=1028&upmidCd=0102#a110dc6b3a1678330158473e0d0ffbf0=3)
      - [KRX Market Data System](http://data.krx.co.kr/contents/MDC/MDI/mdiLoader/index.cmd?menuId=MDC0201010106)

## Data Load

In [2]:
list_file = glob(f"{data_path}KRX_KOSPI200_components/KRX_KOSPI200_components*.csv")
print(list_file)

['../data/KRX_KOSPI200_components\\KRX_KOSPI200_components_20100630.csv', '../data/KRX_KOSPI200_components\\KRX_KOSPI200_components_20101230.csv', '../data/KRX_KOSPI200_components\\KRX_KOSPI200_components_20110630.csv', '../data/KRX_KOSPI200_components\\KRX_KOSPI200_components_20111230.csv', '../data/KRX_KOSPI200_components\\KRX_KOSPI200_components_20120629.csv', '../data/KRX_KOSPI200_components\\KRX_KOSPI200_components_20121228.csv', '../data/KRX_KOSPI200_components\\KRX_KOSPI200_components_20130628.csv', '../data/KRX_KOSPI200_components\\KRX_KOSPI200_components_20131230.csv', '../data/KRX_KOSPI200_components\\KRX_KOSPI200_components_20140630.csv', '../data/KRX_KOSPI200_components\\KRX_KOSPI200_components_20141230.csv', '../data/KRX_KOSPI200_components\\KRX_KOSPI200_components_20150630.csv', '../data/KRX_KOSPI200_components\\KRX_KOSPI200_components_20151230.csv', '../data/KRX_KOSPI200_components\\KRX_KOSPI200_components_20160630.csv', '../data/KRX_KOSPI200_components\\KRX_KOSPI200_com

## df_components로 병합

In [3]:
df_components = []
for f in list_file:
    new = pd.read_csv(f, encoding="cp949")
    df_components.append(new)

# 병합
df_components = pd.concat(df_components)
# 확인
df_components

,종목코드,종목명,종가,대비,등락률,상장시가총액
0,5930,삼성전자,1058000,-3000,-0.28,155842699.0
1,5380,현대차,213000,0,0.00,46918890.0
2,5490,POSCO,380000,-5500,-1.43,33130997.0
3,12330,현대모비스,292000,1500,0.52,28424408.0
4,270,기아차,66700,500,0.76,26934445.0
...,...,...,...,...,...,...
195,284740,쿠쿠홈시스,28300,-250,-0.88,634976.0
196,105630,한세실업,16950,-450,-2.59,678000.0
197,192080,더블유게임즈,38400,-350,-0.90,705581.0
198,192820,코스맥스,56500,-4100,-6.77,641247.0


## 전처리
- df_components 전처리

In [4]:
# 중복 제거
df_components = df_components.drop_duplicates(["종목코드"], keep="last")
# 필요한 컬럼만
df_components = df_components[["종목코드", "종목명"]]
# 정렬
df_components = df_components.sort_values(by="종목코드", ascending=True)
# 종목코드 6자리로 반환하기
df_components["종목코드"] = df_components["종목코드"].apply(m.six_digit)
# 확인
df_components

,종목코드,종목명
179,000020,동화약품
27,000030,우리은행
189,000050,경방
74,000060,메리츠화재
191,000070,삼양홀딩스
...,...,...
41,377300,카카오페이
171,381970,케이카
62,383220,F&F
193,383800,LX홀딩스


## 영속화
- df_components
- KRX_KOSPI200_components.csv

In [5]:
m.DfPrst(df_components, fp_components)

['../data/components_list.csv']


# ```stockPrice.csv```
  - 개요
    - KRX KOSPI200 구성종목 주식가격 데이터
  - Data Collection Method
    - [FinanceDataReader](https://github.com/financedata-org/FinanceDataReader)

## Data Load

In [6]:
df_components = m.DataLoad(fp_components)
m.Check_df(df_components)

Mem. usage decreased to  0.00 Mb (24.4% reduction)


NameError: name '종목코드' is not defined

In [3]:
if glob(fp_components):
    df_components = pd.read_csv(fp_components, index_col=False)
df_components = df_components

# 종목코드 자리수
df_components["종목코드"] = df_components["종목코드"].apply(m.six_digit)
df_components.head()

,종목코드,종목명
0,000020,동화약품
1,000030,우리은행
2,000050,경방
3,000060,메리츠화재
4,000070,삼양홀딩스


## 코스피200 종목의 주가정보 가져오기

In [4]:
# krx 상장종목 모두 가져오기
df_krx = fdr.StockListing("KRX")
df_krx.head(2)

,Code,ISU_CD,Name,Market,Dept,Close,ChangeCode,Changes,ChagesRatio,Open,High,Low,Volume,Amount,Marcap,Stocks,MarketId
0,005930,KR7005930003,삼성전자,KOSPI,,63800,1,300,0.47,63900,64000,63000,15194598,967336146677,380872126690000,5969782550,STK
1,373220,KR7373220003,LG에너지솔루션,KOSPI,,535000,1,2000,0.38,533000,541000,530000,311193,166315331500,125190000000000,234000000,STK


In [5]:
# krx 전체 상장 종목중 KOSPI 상장 종목 추출하기
df_kospi = df_krx[df_krx["Market"] == "KOSPI"]
df_kospi.head(2)

,Code,ISU_CD,Name,Market,Dept,Close,ChangeCode,Changes,ChagesRatio,Open,High,Low,Volume,Amount,Marcap,Stocks,MarketId
0,005930,KR7005930003,삼성전자,KOSPI,,63800,1,300,0.47,63900,64000,63000,15194598,967336146677,380872126690000,5969782550,STK
1,373220,KR7373220003,LG에너지솔루션,KOSPI,,535000,1,2000,0.38,533000,541000,530000,311193,166315331500,125190000000000,234000000,STK


In [6]:
# KOSPI 200 기업 추출하기
df_kospi200 = df_kospi[df_kospi["Code"].isin(df_components["종목코드"].values)]
df_kospi200 = df_kospi200.sort_values(by="Code", ascending=True)
df_kospi200.head(2)

,Code,ISU_CD,Name,Market,Dept,Close,ChangeCode,Changes,ChagesRatio,Open,High,Low,Volume,Amount,Marcap,Stocks,MarketId
735,000020,KR7000020008,동화약품,KOSPI,,9400,2,-80,-0.84,9470,9500,9390,52676,496658090,262555818000,27931470,STK
653,000050,KR7000050005,경방,KOSPI,,10950,1,70,0.64,10920,10980,10830,4765,52025920,300197206500,27415270,STK


In [7]:
# 코스피 200 기업 종목코드 추출
list_code = df_kospi200["Code"].values.tolist()
print(len(list_code))

# 코스피 200 기업 종목명 추출
list_name = df_kospi200["Name"].values.tolist()
print(len(list_name))

321
321


In [13]:
# 모든 종목의 주가를 저장
df = []
for i in tqdm(range(len(list_code))):
    stock = fdr.DataReader(list_code[i], "2011")
    df.append(stock)

# 데이터프레임에 종목코드 추가
for i in tqdm(range(len(list_code))):
    df[i]["Code"] = list_code[i]

# 데이터프레임에 종목명 추가
for i in tqdm(range(len(list_name))):
    df[i]["Name"] = list_name[i]

# 모든 데이터프레임을 병합
df_stock_raw = pd.concat(df)

# 인덱스를 제거하고 컬럼에 추가
df_stock_raw = df_stock_raw.reset_index()

# (선택) 실행 시간이 오래걸려서 백업
with open("df_stock_raw.pickle", "wb") as f:
    pickle.dump(df_stock_raw, f)

# 실행 시간이 오래걸려서 백업
df_stock = df_stock_raw.copy()

# 병합한 데이터를 미리보기
df_stock.sample(5)

100%|██████████| 321/321 [00:00<00:00, 3443.68it/s]


,Date,Open,High,Low,Close,Volume,Change,Code,Name
30223,2012-09-13,7480,7708,7458,7687,11351,0.033616,000230,일동홀딩스
687297,2013-02-22,71700,72000,70300,71000,103957,-0.013889,071840,롯데하이마트
747249,2020-07-08,12400,12400,12000,12150,78150,-0.016194,093050,LF
486599,2022-04-05,38150,38250,37850,38050,81785,0.001316,017800,현대엘리베이
601470,2011-09-30,28701,29765,28199,28782,63071,0.001392,035720,카카오


In [14]:
# (선택) 백업한 피클 불러오기
with open("df_stock_raw.pickle", "rb") as f:
    df_stock = pickle.load(f)
df_stock

,Date,Open,High,Low,Close,Volume,Change,Code,Name
0,2011-01-03,5320,5370,5220,5280,78725,-0.001890,000020,동화약품
1,2011-01-04,5250,5330,5230,5300,58101,0.003788,000020,동화약품
2,2011-01-05,5300,5350,5220,5240,78870,-0.011321,000020,동화약품
3,2011-01-06,5240,5250,5070,5110,174303,-0.024809,000020,동화약품
4,2011-01-07,5110,5150,5080,5090,36832,-0.003914,000020,동화약품
...,...,...,...,...,...,...,...,...,...
863271,2023-01-30,37850,38150,37250,37550,367640,-0.003979,402340,SK스퀘어
863272,2023-01-31,37550,37850,36250,36250,959658,-0.034621,402340,SK스퀘어
863273,2023-02-01,36400,36950,36300,36950,311314,0.019310,402340,SK스퀘어
863274,2023-02-02,37400,37500,36750,37000,605459,0.001353,402340,SK스퀘어


## 전처리

In [15]:
# 컬럼명 변경하기
df_stock.columns = ["연_월_일", "시가", "고가", "저가", "종가", "거래량", "등락률", "종목코드", "종목명"]
# 컬럼 순서
list_colOrder = ["종목코드", "종목명", "연_월_일", "시가", "고가", "저가", "종가", "거래량", "등락률"]
df_stock = df_stock[list_colOrder]
df_stock.head(2)

,종목코드,종목명,연_월_일,시가,고가,저가,종가,거래량,등락률
0,000020,동화약품,2011-01-03,5320,5370,5220,5280,78725,-0.001890
1,000020,동화약품,2011-01-04,5250,5330,5230,5300,58101,0.003788


In [16]:
m.DerivedCol_Date(df_stock, col_YMD="연_월_일", inplace=True)

,종목코드,종목명,연_월_일,시가,고가,저가,종가,거래량,등락률,연,분기,월,일,연_분기,연_월,분기_월,월_일,연_분기_월
0,000020,동화약품,2011-01-03,5320,5370,5220,5280,78725,-0.001890,2011,1,1,3,2011-1,2011-1,1-1,1-3,2011-1-1
1,000020,동화약품,2011-01-04,5250,5330,5230,5300,58101,0.003788,2011,1,1,4,2011-1,2011-1,1-1,1-4,2011-1-1
2,000020,동화약품,2011-01-05,5300,5350,5220,5240,78870,-0.011321,2011,1,1,5,2011-1,2011-1,1-1,1-5,2011-1-1
3,000020,동화약품,2011-01-06,5240,5250,5070,5110,174303,-0.024809,2011,1,1,6,2011-1,2011-1,1-1,1-6,2011-1-1
4,000020,동화약품,2011-01-07,5110,5150,5080,5090,36832,-0.003914,2011,1,1,7,2011-1,2011-1,1-1,1-7,2011-1-1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
863271,402340,SK스퀘어,2023-01-30,37850,38150,37250,37550,367640,-0.003979,2023,1,1,30,2023-1,2023-1,1-1,1-30,2023-1-1
863272,402340,SK스퀘어,2023-01-31,37550,37850,36250,36250,959658,-0.034621,2023,1,1,31,2023-1,2023-1,1-1,1-31,2023-1-1
863273,402340,SK스퀘어,2023-02-01,36400,36950,36300,36950,311314,0.019310,2023,1,2,1,2023-1,2023-2,1-2,2-1,2023-1-2
863274,402340,SK스퀘어,2023-02-02,37400,37500,36750,37000,605459,0.001353,2023,1,2,2,2023-1,2023-2,1-2,2-2,2023-1-2


### MinMaxScaling
- 일반적인 MinMaxScaling은 컬럼의 Min과 Max를 기준으로 스케일링되지만
- 이 분석의 경우에는 적절하지 못하므로 개별 종목의 Min과 Max를 기준으로 스케일링을 진행함.
- 액면분할과 액면병합을 고려하지 않아 한계점이 존재함.

In [18]:
df_stock = m.DerivedCol_Groupby_MinMaxScaler(df_stock, ["종목코드", "종목명"],["시가", "종가", "거래량"])
df_stock

,종목코드,종목명,연_월_일,시가,고가,저가,종가,거래량,등락률,시가_mmscl,종가_mmscl,거래량_mmscl
0,000020,동화약품,2011-01-03,5320,5370,5220,5280,78725,-0.001890,0.049734,0.046130,0.003963
1,000020,동화약품,2011-01-04,5250,5330,5230,5300,58101,0.003788,0.047077,0.046893,0.002824
2,000020,동화약품,2011-01-05,5300,5350,5220,5240,78870,-0.011321,0.048975,0.044605,0.003971
3,000020,동화약품,2011-01-06,5240,5250,5070,5110,174303,-0.024809,0.046697,0.039649,0.009242
4,000020,동화약품,2011-01-07,5110,5150,5080,5090,36832,-0.003914,0.041762,0.038887,0.001649
...,...,...,...,...,...,...,...,...,...,...,...,...
863271,402340,SK스퀘어,2023-01-30,37850,38150,37250,37550,367640,-0.003979,0.121393,0.126136,0.019279
863272,402340,SK스퀘어,2023-01-31,37550,37850,36250,36250,959658,-0.034621,0.115423,0.096591,0.069919
863273,402340,SK스퀘어,2023-02-01,36400,36950,36300,36950,311314,0.019310,0.092537,0.112500,0.014461
863274,402340,SK스퀘어,2023-02-02,37400,37500,36750,37000,605459,0.001353,0.112438,0.113636,0.039622


## 영속화

In [19]:
m.DfPrst(df_stock, fp_stock)

['../data/stockPrice.parquet']
